# Aprendizaje automático relacional

#### Fernando Jesús Fernández Gallardo
#### Carmen Galván López

## Preparación

#### Imports y variables globales

In [ ]:
from pandas import read_csv
import numpy as np
import networkx as nx
from sklearn import preprocessing, model_selection, naive_bayes
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.svm import SVC

semilla = 86
test_size= .33

#### Lectura y procesamiento inicial de los datos brutos

In [ ]:
#Leemos los archivos
aristas = read_csv('data/political-books-edges.csv')
vertices = read_csv('data/political-books-nodes.csv')

#Borramos la columna ID
del(vertices['Id'])

#Mostramos las primeras 35 filas
vertices.head(35)

#### Selección y validación de los datos brutos

In [ ]:
"""
Comprobamos que el dataset es válido verificando que no existen duplicados
"""
if len(vertices) != len(set(vertices['Label'])):
    raise ValueError("El dataset no es válido ya que contiene duplicados")
"""
La mejor forma de identificar cada uno de los elementos que forma parte
del conjunto de entrenamiento es el nombre del propio libro (que en el dataset
se llama 'Label') en vez del ID o cualquier otro tipo de indentificador más
complejo. De esta forma, también es más fácil identificar elementos duplicados
(si los hubiera)
"""
atributos = vertices['Label']
"""
Nuestro objetivo es predecir la ideología política del autor basándonos en
sus obras, por lo que el objetivo que perseguimos en nuestro modelo
es el de la ideología política
"""
objetivo = vertices['political_ideology']

## Inicio del entrenamiento
#### Codificación del objetivo

In [ ]:

"""
Para poder trabajar con los datos que tenemos, necesitamos convertirlos en un formato que sklearn pueda "entender".
Debemos de hacer que nuestros datos "planos" sean para sklearn objetos "comparables", dependiendo del tipo de
ordenación que nosotros veamos más apropiada para el método en cuestión
(de una manera similar hacemos en Java cuando implementamos la interfaz 'Comparable' y el método compareTo)

El codificador adecuado para la variable objetivo es LabelEncoder, que trabaja
con una lista o array unidimensional de sus valores y admite cadenas

"""
# Codificadores
codificador_atributos = preprocessing.LabelEncoder()
codificador_objetivo = preprocessing.LabelEncoder()
# Datos codificados
atributos_codificados = codificador_atributos.fit_transform(atributos)
objetivo_codificado = codificador_objetivo.fit_transform(objetivo)

#### División en conjunto de entrenamiento y conjunto de prueba

Partimos el atributo y el objetivo en dos, de entrenamiento y de prueba. Esto se hace para evitar el sobreajuste, que es cuando el modelo se adapta demasiado a los datos de entrenamiento y pierde capacidad de generalizar a nuevos casos.

Reservamos con `test_size` el 33% del total de los datos para la posterior prueba. El resto se usa para el entrenamiento.

Con `stratify` indicamos la división que debe mantener la proporción de clases en ambos conjuntos, es decir, que si hay un 40% de conservadores, un 30% de neutrales y un 30% de liberales en el conjunto original, también haya esa proporción en el conjunto de entrenamiento y en el de prueba. Esto se hace para evitar sesgos.

In [ ]:
(atributos_entrenamiento,
 atributos_prueba,
 objetivo_entrenamiento,
 objetivo_prueba) = model_selection.train_test_split(
        atributos_codificados,
        objetivo_codificado,
        # Valor de la semilla aleatoria para que el muestreo sea reproducible a pesar de ser aleatorio
        random_state=semilla,
        test_size=test_size,
        stratify=objetivo_codificado
)

## Naive Bayes
#### No relacional

In [ ]:
"""
El método reshape solo cambia la forma del array, pero no su contenido.
El clasificador MultinomialNB de sklearn espera un array 2D porque puede manejar
múltiples características por muestra, pero en nuestro caso solo tenemos una característica por muestra,
que es el nombre del libro.
"""
atr_cod_reshaped = atributos_codificados.reshape(-1, 1)
atr_pr_reshaped = atributos_prueba.reshape(-1, 1)
atr_ent_reshaped = atributos_entrenamiento.reshape(-1, 1)

## Explicación de Bing, borrar antes de entregar:

```
Lo que haces con ese código es crear y entrenar un clasificador de Naive Bayes no relacional, que es un tipo de modelo que asume que los atributos son independientes entre sí dada la clase. Es decir, que la probabilidad de que un libro tenga un cierto título no depende de la probabilidad de que tenga otra característica, como el autor o el género. Esto simplifica el cálculo de las probabilidades, pero puede ser una suposición muy fuerte en algunos casos.

El clasificador que usas es el MultinomialNB, que es adecuado para atributos discretos que representan frecuencias o conteos. Por ejemplo, si los títulos de los libros se representan como vectores de palabras, donde cada posición indica cuántas veces aparece una palabra en el título. El parámetro alpha es el suavizado, que es una técnica para evitar que las probabilidades sean cero cuando hay palabras que no aparecen en el conjunto de entrenamiento. Un valor típico es 1, pero puedes probar otros valores para ver cómo afectan al rendimiento del modelo.

El método fit es el que entrena el modelo con los datos de entrenamiento. Le pasas los atributos y los objetivos codificados, y el modelo aprende las probabilidades de cada clase y de cada atributo dado cada clase. Estas probabilidades se almacenan en los atributos class_log_prior_ y feature_log_prob_ del modelo.

El bucle for que haces al final es para mostrar la cantidad de ejemplos y el logaritmo de la probabilidad de cada clase. El logaritmo se usa para evitar problemas de desbordamiento o subdesbordamiento cuando se multiplican muchas probabilidades pequeñas. La clase 0 corresponde a conservador, la 1 a neutral y la 2 a liberal.
```

In [ ]:
suavizado = 1

clasif_NB = naive_bayes.MultinomialNB(alpha=suavizado)
clasif_NB.fit(atr_ent_reshaped, objetivo_entrenamiento)

#Calculamos la cantidad de ejemplos para cada clase y los logaritmos
for clase, cantidad_ejemplos_clase, log_probabilidad_clase in zip(
        clasif_NB.classes_, clasif_NB.class_count_, clasif_NB.class_log_prior_):
    print(f"Cantidad de ejemplos para la clase {0}: {1}", clase, cantidad_ejemplos_clase)
    print(f"Logaritmo de la probabilidad aprendida para la clase {0}: {1}", clase, log_probabilidad_clase)

## EXPLICACIÓN DE BING, BORRAR

```
Lo que haces con ese código es evaluar el rendimiento del modelo de Naive Bayes que has entrenado. Para ello, usas varias técnicas:

- Primero, usas el método predict para obtener las predicciones del modelo para los datos de prueba. Le pasas los atributos de prueba codificados y te devuelve un arreglo con los objetivos predichos. Estos los puedes comparar con los objetivos reales para ver cuántos aciertos y errores tiene el modelo.
- Luego, usas el método score para obtener la precisión del modelo, que es la proporción de aciertos sobre el total de casos. Le pasas los atributos y los objetivos de prueba codificados y te devuelve un valor entre 0 y 1, donde 1 significa que el modelo acierta todos los casos y 0 que falla todos.
- Después, usas la técnica de cross validation, que es una forma de estimar la precisión del modelo usando diferentes particiones de los datos. Creas un objeto ShuffleSplit que define cómo se van a dividir los datos en cada iteración. En este caso, haces 10 iteraciones, reservando el 33% de los datos para la prueba y usando la misma semilla aleatoria que antes. Luego, usas la función cross_val_score para obtener la precisión del modelo en cada iteración. Le pasas el modelo, los atributos y los objetivos codificados de todo el conjunto de datos y el objeto ShuffleSplit. Te devuelve un arreglo con las precisiones obtenidas en cada iteración.
- Finalmente, usas la función np.mean para obtener la media de las precisiones obtenidas con cross validation. Esto te da una idea de cómo se comporta el modelo en promedio con diferentes conjuntos de datos.

Con todo esto, puedes tener una medida más robusta y confiable del rendimiento del modelo de Naive Bayes. Espero haberte ayudado a entender mejor lo que está sucediendo en tu código. 😊
```

In [ ]:
"""
ShuffleSplit es necesario para la CrossValidation
"""
cv = ShuffleSplit(n_splits=10, test_size=test_size, random_state=semilla)


#Probamos la predicción con los atributos de prueba
print('Predicción con Naive Bayes: ', clasif_NB.predict(atr_pr_reshaped))
#Hacemos el score con naive bayes
print('Precisión con Naive Bayes: ', clasif_NB.score(atr_pr_reshaped, objetivo_prueba))
#Hacemos el score con cross validation
print('Precisión con cross validation: ', cross_val_score(clasif_NB, atr_cod_reshaped, objetivo_codificado, cv=cv))
#Hacemos la media de score de cross validation, ya que al final es lo que nos interesa
print(f'Media de precisión: {0}', np.mean(cross_val_score(clasif_NB, atr_cod_reshaped, objetivo_codificado, cv=cv)))

## SVC

In [ ]:
#Vamos a utilizar este modelo ya que nos viene bien al tener un dataset pequeño. Este modelo aumenta
#el tiempo de entrenamiento cuadrátricamente con el número de ejemplos

classif_SVC = SVC().fit(atr_ent_reshaped, objetivo_entrenamiento)


#Probamos la predicción con los atributos de prueba
print('Predicción SVC: ', classif_SVC.predict(atr_pr_reshaped))
#Hacemos el score con kNN
print('Precisión SVC: ', classif_SVC.score(atr_pr_reshaped, objetivo_prueba))
#Hacemos el score con cross validation
print(f'Precisión cross validation: {0}', np.mean(cross_val_score(classif_SVC, atr_cod_reshaped, objetivo_codificado, cv=cv)))

## Sacar métricas relacionales